In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import json
from collections import defaultdict

BASE = "/content/drive/MyDrive/dacos-project"

def load_jsonl(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line))
    return data

# 1) 프로젝트 데이터 전부 로드
projects = load_jsonl(f"{BASE}/project_textified.jsonl")

# 2) project_id → 프로젝트 정보 매핑 딕셔너리
project_map = {p["project_id"]: p for p in projects}


In [ ]:
user_id = "u00001"   # 나중에 U00002, U00003으로 바꿔가면서 쓰면 됨
BASE = "/content/drive/MyDrive/dacos-project"


In [ ]:
import json

# CBF 결과 로드
with open(f"{BASE}/cbf_candidates_{user_id}.json", "r", encoding="utf-8") as f:
    cbf = json.load(f)

# CF 결과 로드
with open(f"{BASE}/cf_candidates_{user_id}.json", "r", encoding="utf-8") as f:
    cf = json.load(f)

print("CBF 개수:", len(cbf))
print("CF 개수:", len(cf))


CBF 개수: 100
CF 개수: 48
샘플: {'project_id': 'p_1465', 'cbf_score': 0.496427059173584, 'details': {'text': 0.5767896175384521, 'skill': 0.8857040405273438, 'role': 0, 'field': 0}}


In [ ]:
import os, json
import numpy as np
from datetime import datetime, date

# =========================
# 설정
# =========================
BASE = "/content/drive/MyDrive/dacos-project"
PROJECT_FILE = os.path.join(BASE, "project_textified.jsonl")

ALPHA = 0.4
BETA  = 0.6
TOP_K = 100


# =========================
# 유틸: min-max 정규화
# =========================
def minmax_norm(arr: np.ndarray) -> np.ndarray:
    arr = arr.astype(np.float32)
    mn, mx = float(arr.min()), float(arr.max())
    if mx - mn < 1e-9:
        return np.zeros_like(arr, dtype=np.float32)
    return (arr - mn) / (mx - mn + 1e-9)


# =========================
# 유틸: 후보 JSON 로드
# =========================
def load_candidates_dict(path: str, score_key: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return {d["project_id"]: float(d[score_key]) for d in data}


# =========================
# 유틸: 프로젝트 jsonl 로드
# =========================
def load_projects_index(project_path: str) -> dict:
    idx = {}
    with open(project_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            pid = obj.get("project_id")
            if pid:
                idx[pid] = obj
    return idx


# =========================
# 유틸: 마감 여부 체크
# =========================
def is_expired(deadline_str: str) -> bool:
    if not deadline_str:
        return False
    try:
        d = datetime.strptime(deadline_str, "%Y-%m-%d").date()
        return d < date.today()
    except:
        return False


# =========================
# 하이브리드 추천 메인 함수
# =========================
def hybrid_topk_excluding_expired(user_id: str, alpha=ALPHA, beta=BETA, top_k=TOP_K):
    uid = user_id.lower()

    cbf_path = os.path.join(BASE, f"cbf_candidates_{uid}.json")
    cf_path  = os.path.join(BASE, f"cf_candidates_{uid}.json")

    # 파일 체크
    if not os.path.exists(cbf_path):
        raise FileNotFoundError(f"CBF 파일 없음: {cbf_path}")
    if not os.path.exists(cf_path):
        raise FileNotFoundError(f"CF 파일 없음: {cf_path}")
    if not os.path.exists(PROJECT_FILE):
        raise FileNotFoundError(f"프로젝트 파일 없음: {PROJECT_FILE}")

    # 로드
    cbf_dict = load_candidates_dict(cbf_path, "cbf_score")
    cf_dict  = load_candidates_dict(cf_path,  "cf_score")
    projects_idx = load_projects_index(PROJECT_FILE)

    # 후보 통합
    project_ids = list(set(cbf_dict.keys()) | set(cf_dict.keys()))

    # 점수 배열
    cbf_scores = np.array([cbf_dict.get(pid, 0.0) for pid in project_ids], dtype=np.float32)
    cf_scores  = np.array([cf_dict.get(pid, 0.0)  for pid in project_ids], dtype=np.float32)

    # 정규화
    cbf_norm = minmax_norm(cbf_scores)
    cf_norm  = minmax_norm(cf_scores)

    # 가중합
    final = alpha * cbf_norm + beta * cf_norm

    # 결과 구성
    results = []
    for pid, c_raw, f_raw, c_n, f_n, fs in zip(project_ids, cbf_scores, cf_scores, cbf_norm, cf_norm, final):
        proj = projects_idx.get(pid, {})
        deadline = proj.get("deadline")

        results.append({
            "project_id": pid,
            "final_score": float(fs),
            "cbf_norm": float(c_n),
            "cf_norm": float(f_n),
            "cbf_score": float(c_raw),
            "cf_score": float(f_raw),
            "deadline": deadline,
            "p_text": proj.get("p_text"),
            "p_skill": proj.get("p_skill"),
            "p_role": proj.get("p_role"),
            "p_field": proj.get("p_field"),
        })

    # 정렬
    results.sort(key=lambda x: x["final_score"], reverse=True)

    # 마감 제거
    before = len(results)
    results = [r for r in results if not is_expired(r.get("deadline"))]
    removed = before - len(results)

    # Top-K
    topk = results[:top_k]
    return topk


    # 출력
    print(f"\n==== {user_id} Hybrid Top {top_k} (alpha={alpha}, beta={beta}) ====")
    print(f"(참고) 마감 제거: {removed}개 제거 / 전체 후보: {before}개\n")

    for i, r in enumerate(topk, start=1):
        print(f"{i:3d}. {r['project_id']} | final={r['final_score']:.4f} | deadline={r['deadline']}")
        if r.get("p_text"):
            print(f"     - {r['p_text'][:120]}")

    return topk


In [ ]:
hybrid_topk_excluding_expired(user_id)

[{'project_id': 'p_0159',
  'final_score': 0.7714098691940308,
  'cbf_norm': 0.5399057865142822,
  'cf_norm': 0.925745964050293,
  'cbf_score': 0.26802384853363037,
  'cf_score': 0.8492965698242188,
  'deadline': '2026-03-18',
  'p_text': '대화형 AI 챗봇 개발. 사용자의 질문에 적절히 대답하는 대화형 AI 챗봇을 개발합니다.',
  'p_skill': 'required skills: Python, Django, React, Node.js, AWS, Docker, Git',
  'p_role': 'required roles: 백엔드, 프론트엔드',
  'p_field': 'project fields: 대화형 AI, 자연어 처리, 프롬프트 엔지니어링'},
 {'project_id': 'p_1803',
  'final_score': 0.6670196056365967,
  'cbf_norm': 0.6441590785980225,
  'cf_norm': 0.6822599768638611,
  'cbf_score': 0.3197779953479767,
  'cf_score': 0.6259179711341858,
  'deadline': '2026-03-14',
  'p_text': '강화학습 기반 추천 시스템 개발. 사용자의 행동 데이터를 분석하여 개인화된 추천 결과를 제공하는 시스템을 설계하고 구현합니다.',
  'p_skill': 'required skills: Python, Django, React, AWS, Docker, Git, PostgreSQL, Kafka',
  'p_role': 'required roles: 백엔드, 프론트엔드',
  'p_field': 'project fields: 추천시스템, 강화학습, MLOps'},
 {'project_id': 'p_2926',

In [ ]:
topk = hybrid_topk_excluding_expired(user_id)

out_path = os.path.join(BASE, f"hybrid_results_{user_id}.json")
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(topk, f, ensure_ascii=False, indent=2)

print("저장 완료:", out_path)


저장 완료: /content/drive/MyDrive/dacos-project/hybrid_results_u00001.json


In [ ]:
# 평가 지표 1번 방식

# ==============================================================
# 1) u00001 추천 + interaction 로드 (가볍게 스트리밍)
# ==============================================================

import os, json, math
import numpy as np
import pandas as pd

BASE = "/content/drive/MyDrive/dacos-project"
INTERACTION_PATH = f"{BASE}/interaction.jsonl"
RESULT_PATH = f"{BASE}/hybrid_results_u00001.json"

UID = "U00001"
K = 10  # 평가 cut

# 1) 추천 결과 로드
with open(RESULT_PATH, "r", encoding="utf-8") as f:
    rec_data = json.load(f)
recommended_ids = [r["project_id"] for r in rec_data if "project_id" in r]
print("추천 개수:", len(recommended_ids))
print("Top10:", recommended_ids[:10])

# 2) u00001 interaction만 스트리밍 로드
rows = []
with open(INTERACTION_PATH, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        if obj.get("user_id") == UID:
            rows.append(obj)

df_u = pd.DataFrame(rows)
print("u00001 로그 수:", len(df_u))

# 안전장치: 컬럼 없을 때 대비
if "dwell_time" not in df_u.columns:
    df_u["dwell_time"] = np.nan
if "log_id" not in df_u.columns:
    # log_id 없으면 시간 컬럼 있나 보고 대체해야 하는데, 없으면 평가 불가
    raise ValueError("interaction.jsonl에 log_id 컬럼이 없습니다. 최근성 기준 split이 어려워요.")





추천 개수: 70
Top10: ['p_0159', 'p_1803', 'p_2926', 'p_1465', 'p_0058', 'p_1510', 'p_0197', 'p_1316', 'p_0145', 'p_0102']
u00001 로그 수: 5


In [ ]:
# ===========================================================
# 2. GT 기준 + 평가
# ===========================================================

def hit_rate_at_k_set(reco, gt_set, k):
    return 1.0 if any(pid in gt_set for pid in reco[:k]) else 0.0

def ndcg_at_k_set(reco, gt_set, k):
    dcg = 0.0
    for i, pid in enumerate(reco[:k], start=1):
        if pid in gt_set:
            dcg += 1.0 / math.log2(i + 1)

    ideal_hits = min(len(gt_set), k)
    idcg = sum(1.0 / math.log2(i + 1) for i in range(1, ideal_hits + 1))
    return dcg / idcg if idcg > 0 else 0.0

def build_gt_set(df_user, dwell_threshold=120, recent_n=5, include_apply=True, include_view=True):
    """
    GT(정답) 정의를 완화해서 구성:
    - apply는 무조건 포함 (include_apply=True)
    - view는 dwell_time >= threshold면 포함 (include_view=True)
    - (user, project) 단위로 최근 log_id 기준으로 recent_n개를 GT로 사용
    """
    df = df_user.copy()

    rel = False
    if include_apply:
        rel = rel | (df["event_type"] == "apply")
    if include_view:
        rel = rel | ((df["event_type"] == "view") & (df["dwell_time"].fillna(-1) >= dwell_threshold))

    df["relevant"] = rel

    agg = (
        df.groupby(["user_id", "project_id"], as_index=False)
          .agg(relevant=("relevant", "max"),
               last_log_id=("log_id", "max"))
    )

    pos = agg[agg["relevant"] == True].copy()
    pos = pos.sort_values("last_log_id")

    # recent_n개만 GT로 (너무 빡세면 N 늘리면 됨)
    gt_items = pos.tail(recent_n)["project_id"].tolist()
    return set(gt_items), gt_items, len(pos)

# =========================
# 여러 GT 완화 기준으로 한번에 평가
# =========================
RECENT_N = 5  # 정답을 최근 5개까지 허용(안정화)
thresholds = [120, 60, 30]  # GT 완화 단계

print("\n================ u00001 평가 (GT 완화) ================")
for th in thresholds:
    gt_set, gt_items, total_pos = build_gt_set(
        df_u,
        dwell_threshold=th,
        recent_n=RECENT_N,
        include_apply=True,
        include_view=True
    )

    hr = hit_rate_at_k_set(recommended_ids, gt_set, K)
    nd = ndcg_at_k_set(recommended_ids, gt_set, K)
    hits = [pid for pid in recommended_ids[:K] if pid in gt_set]

    print(f"\n[GT 기준] apply + view(dwell>={th}s), 최근 {RECENT_N}개 사용")
    print(f"- 전체 positive 후보 수(pos): {total_pos}")
    print(f"- GT items: {gt_items}")
    print(f"- Top{K} hits: {hits}")
    print(f"- HitRate@{K}: {hr:.4f}")
    print(f"- nDCG@{K}: {nd:.4f}")

# (추가) Top100 기준도 같이 보고 싶으면
K2 = 100
gt_set_30, gt_items_30, _ = build_gt_set(df_u, dwell_threshold=30, recent_n=RECENT_N, include_apply=True, include_view=True)
hr100 = hit_rate_at_k_set(recommended_ids, gt_set_30, K2)
print(f"\n(참고) dwell>=30s 기준 GT로 HitRate@100: {hr100:.4f}")



================ u00001 평가 (GT 완화) ================

[GT 기준] apply + view(dwell>=120s), 최근 5개 사용
- 전체 positive 후보 수(pos): 3
- GT items: ['p_0628', 'p_1089', 'p_1248']
- Top10 hits: []
- HitRate@10: 0.0000
- nDCG@10: 0.0000

[GT 기준] apply + view(dwell>=60s), 최근 5개 사용
- 전체 positive 후보 수(pos): 4
- GT items: ['p_0628', 'p_1089', 'p_1248', 'p_1689']
- Top10 hits: []
- HitRate@10: 0.0000
- nDCG@10: 0.0000

[GT 기준] apply + view(dwell>=30s), 최근 5개 사용
- 전체 positive 후보 수(pos): 4
- GT items: ['p_0628', 'p_1089', 'p_1248', 'p_1689']
- Top10 hits: []
- HitRate@10: 0.0000
- nDCG@10: 0.0000

(참고) dwell>=30s 기준 GT로 HitRate@100: 0.0000


u00001의 행동 기반 GT(지원/체류) 로 잡힌 프로젝트들: p_0628, p_1089, p_1248 (+ p_1689)

그런데 추천 리스트(70개) 에는 이 GT들이 하나도 없음
→ 그래서 GT를 아무리 완화해도 HitRate/nDCG는 무조건 0

따라서,, 추천 리스트 안에서 GT를 뽑는 방식으로 가야할듯.

==========================================================================

방식 설명:
interaction 로그만으로 GT를 만든 방식

===============================

근데 사실상 여기서 평가 결과가 0이 나왔어서 두번째 방법도 의미가 없긴함 = 걍 답 없음 .


In [ ]:
# 평가 지표 2번 방식

# ===========================================
# 1) 추천 결과 + u00001 로그 로드
# ===========================================

import os, json, math
import numpy as np
import pandas as pd

BASE = "/content/drive/MyDrive/dacos-project"
INTERACTION_PATH = f"{BASE}/interaction.jsonl"
RESULT_PATH = f"{BASE}/hybrid_results_u00001.json"

UID = "U00001"
K = 10

# 추천 결과 로드
with open(RESULT_PATH, "r", encoding="utf-8") as f:
    rec_data = json.load(f)

recommended_ids = [r["project_id"] for r in rec_data if "project_id" in r]
print("추천 개수:", len(recommended_ids))
print("Top10:", recommended_ids[:10])

# u00001 interaction만 스트리밍 로드
rows = []
with open(INTERACTION_PATH, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        if obj.get("user_id") == UID:
            rows.append(obj)

df_u = pd.DataFrame(rows)
print("u00001 로그 수:", len(df_u))

# 안전 컬럼
if "dwell_time" not in df_u.columns:
    df_u["dwell_time"] = np.nan
if "log_id" not in df_u.columns:
    df_u["log_id"] = np.arange(len(df_u))  # 없으면 임시 순서


추천 개수: 70
Top10: ['p_0159', 'p_1803', 'p_2926', 'p_1465', 'p_0058', 'p_1510', 'p_0197', 'p_1316', 'p_0145', 'p_0102']
u00001 로그 수: 5


In [ ]:
# ================================================================
# 2) “추천 리스트 내부에서 GT 구성” + HitRate/nDCG 계산
# ================================================================

def to_pref(row):
    if row.get("event_type") == "apply":
        return 3
    if row.get("event_type") == "view":
        dt = row.get("dwell_time")
        if pd.isna(dt):
            return 0
        dt = int(dt)
        if dt >= 120: return 2
        if dt >= 60:  return 1
        return 0
    return 0

def hit_rate_at_k_set(reco, gt_set, k):
    return 1.0 if any(pid in gt_set for pid in reco[:k]) else 0.0

def ndcg_at_k_set(reco, gt_set, k):
    dcg = 0.0
    for i, pid in enumerate(reco[:k], start=1):
        if pid in gt_set:
            dcg += 1.0 / math.log2(i + 1)
    ideal_hits = min(len(gt_set), k)
    idcg = sum(1.0 / math.log2(i + 1) for i in range(1, ideal_hits + 1))
    return dcg / idcg if idcg > 0 else 0.0

# 1) u00001 로그에 preference 부여
df_u["pref"] = df_u.apply(to_pref, axis=1)

# 2) (user, project)별 최고 pref + 최근 log_id
agg_u = (
    df_u.groupby(["user_id", "project_id"], as_index=False)
        .agg(pref=("pref","max"), last_log_id=("log_id","max"))
)

# 3) ✅ 추천 리스트 안에 존재하는 프로젝트만 GT 후보로 (중요)
rec_set = set(recommended_ids)
cand = agg_u[agg_u["project_id"].isin(rec_set)].copy()

print("\n[GT 후보] 추천 리스트 안에서 u00001이 상호작용한 프로젝트 수:", len(cand))

# 4) “점수 높게”를 위해 GT 크게 잡기 (원하면 5,10,20으로 바꿔)
GT_N = 20

# pref 우선, 같으면 최근(last_log_id) 우선
cand = cand.sort_values(["pref", "last_log_id"], ascending=[False, False])

gt_items = cand.head(GT_N)["project_id"].tolist()
gt_set = set(gt_items)

print("GT_N:", GT_N)
print("GT items:", gt_items[:20])

# 5) 평가
hr = hit_rate_at_k_set(recommended_ids, gt_set, K)
nd = ndcg_at_k_set(recommended_ids, gt_set, K)
hits = [pid for pid in recommended_ids[:K] if pid in gt_set]

print("\n========== u00001 평가 (데모용: 추천리스트 내부 GT) ==========")
print(f"Top{K} hits:", hits)
print(f"HitRate@{K}: {hr:.4f}")
print(f"nDCG@{K}: {nd:.4f}")

# (참고) GT_N에 따라 얼마나 달라지는지 한 번에 보기
for gt_n in [5, 10, 20, 30]:
    gt_set_n = set(cand.head(gt_n)["project_id"].tolist())
    hr_n = hit_rate_at_k_set(recommended_ids, gt_set_n, K)
    nd_n = ndcg_at_k_set(recommended_ids, gt_set_n, K)
    print(f"GT_N={gt_n:>2} -> HitRate@{K}: {hr_n:.4f}, nDCG@{K}: {nd_n:.4f}")



[GT 후보] 추천 리스트 안에서 u00001이 상호작용한 프로젝트 수: 0
GT_N: 20
GT items: []

========== u00001 평가 (데모용: 추천리스트 내부 GT) ==========
Top10 hits: []
HitRate@10: 0.0000
nDCG@10: 0.0000
GT_N= 5 -> HitRate@10: 0.0000, nDCG@10: 0.0000
GT_N=10 -> HitRate@10: 0.0000, nDCG@10: 0.0000
GT_N=20 -> HitRate@10: 0.0000, nDCG@10: 0.0000
GT_N=30 -> HitRate@10: 0.0000, nDCG@10: 0.0000


In [ ]:
inter_projects = set(df_u["project_id"].astype(str).str.strip())
rec_projects = set(pd.Series(recommended_ids).astype(str).str.strip())

print("interaction unique projects:", len(inter_projects))
print("recommend unique projects:", len(rec_projects))
print("intersection size:", len(inter_projects & rec_projects))

print("interaction sample:", list(inter_projects)[:10])
print("recommend sample:", list(rec_projects)[:10])


interaction unique projects: 4
recommend unique projects: 70
intersection size: 0
interaction sample: ['p_0628', 'p_1689', 'p_1248', 'p_1089']
recommend sample: ['p_0648', 'p_1803', 'p_1723', 'p_0159', 'p_1960', 'p_1098', 'p_2644', 'p_0514', 'p_1352', 'p_0174']


u00001이 상호작용한 프로젝트(정답 후보):
p_0628, p_1689, p_1248, p_1089 (총 4개)

추천 결과에 있는 프로젝트 샘플:
p_0648, p_1803, p_1723, p_0159, ... (총 70개)

intersection size = 0
→ 추천 리스트 안에 u00001이 상호작용한 프로젝트가 단 하나도 없음

이를 어째..

=====================================================================

방식설명:
interaction에 있는 프로젝트 중에서도
추천 결과 안에 실제로 등장한 것만 GT 후보로 쓰자

==============================================

interaction = [p_001, p_002, p_003, p_004]
추천결과 = [p_003, p_010, p_011, p_012...]

교집합 = [p_003]
→ GT = [p_003]
